# 環境設定

In [ ]:
# Install HappyML
import os

if not os.path.isdir("HappyML"):
    os.system("git clone https://github.com/cnchi/HappyML.git")

In [ ]:
# Upload Dataset
Dataset_File = "Meals.csv"

if not os.path.isfile(Dataset_File):
  os.system("wget https://raw.githubusercontent.com/cnchi/datasets/master/" + Dataset_File)

In [ ]:
# Customizable Constants
train_size = 0.75

win_size = 2
sample_step = 1
win_moving = 1

data_batch = 10

# 資料集前處理

In [ ]:
# Load Data
import HappyML.preprocessor as pp
import numpy as np

dataset = pp.dataset(file="Meals.csv")

# Remove the date, keep meal only
dataset = pp.onehot_encoder(dataset, columns=[1]).iloc[:, 1:]

In [ ]:
# Keep the One Hot Encoding Mapping
mapper = [s[5:] for s in list(dataset.columns)]

# Convert dataset from DataFrame as NDArray
dataset = dataset.values
train_data, test_data = np.split(dataset, [int(train_size * len(dataset))])

In [ ]:
print(train_data.shape)
print(len(test_data))

(23, 4)
8


# RNN 資料集準備

In [ ]:
# Training Set & Testing Set for RNN
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

train_set = TimeseriesGenerator(
        data=train_data, # 自變數X所在地
        targets=train_data, # 應變數Y所在地
        length=win_size, # 過往視窗大小(=2)
        sampling_rate=sample_step, # 視窗內的取樣頻率(=1)
        stride=win_moving, # 視窗滑動距離(=1)
        batch_size=data_batch) # 一批產生多少筆資料(=10)

test_set = TimeseriesGenerator(
        data=test_data,
        targets=test_data,
        length=win_size,
        sampling_rate=sample_step,
        stride=win_moving,
        batch_size=data_batch)

# 模型建造

In [ ]:
# Create Model
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

# TO-DO: Add more RNN layers & do the validation set
model = Sequential()
model.add(layers.SimpleRNN(10, input_shape=(win_size, dataset.shape[1])))
model.add(layers.Dense(dataset.shape[1], activation="softmax"))
model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["acc"])

# 模型訓練

In [ ]:
# Train Model
# Run multiple times of RNN can raise up the accuracy.

# TO-DO: Find the best epochs
model.fit(train_set, epochs=50)

Epoch 1/50
3/3 [==============================] - 5s 13ms/step - loss: 1.4075 - acc: 0.3333
Epoch 2/50
3/3 [==============================] - 0s 7ms/step - loss: 1.3717 - acc: 0.3333
Epoch 3/50
3/3 [==============================] - 0s 6ms/step - loss: 1.3508 - acc: 0.3333
Epoch 4/50
3/3 [==============================] - 0s 10ms/step - loss: 1.3322 - acc: 0.3333
Epoch 5/50
3/3 [==============================] - 0s 6ms/step - loss: 1.3165 - acc: 0.3333
Epoch 6/50
3/3 [==============================] - 0s 7ms/step - loss: 1.3015 - acc: 0.3333
Epoch 7/50
3/3 [==============================] - 0s 6ms/step - loss: 1.2871 - acc: 0.3333
Epoch 8/50
3/3 [==============================] - 0s 7ms/step - loss: 1.2730 - acc: 0.3333
Epoch 9/50
3/3 [==============================] - 0s 7ms/step - loss: 1.2597 - acc: 0.3333
Epoch 10/50
3/3 [==============================] - 0s 6ms/step - loss: 1.2464 - acc: 0.3333
Epoch 11/50
3/3 [==============================] - 0s 6ms/step - loss: 1.2328 - acc: 0.

# 模型評估

In [ ]:
# Model Evaluation
test_loss, test_acc = model.evaluate(test_set)
print("Loss of Test Set:", test_loss)
print("Accuracy of Test Set:", test_acc)

1/1 [==============================] - 0s 236ms/step - loss: 0.8101 - acc: 0.8333
Loss of Test Set: 0.8101468086242676
Accuracy of Test Set: 0.8333333134651184


# 模型預測

In [ ]:
# Model Prediction
Y_pred = np.argmax(model.predict(test_set), axis=-1)
Y_real = np.argmax(test_set[0][1], axis=-1)

# Convert from number to label
print("Prediction:", [mapper[i] for i in Y_pred])
print("Real Value:", [mapper[i] for i in Y_real])

1/1 [==============================] - 0s 175ms/step
Prediction: ['自助餐', '大魯麵', '自助餐', '黃金炒飯', '大魯麵', '自助餐']
Real Value: ['自助餐', '大魯麵', '自助餐', '黃金炒飯', '大魯麵', '煎餃']


# 使用者資料預測

In [ ]:
# User input
from tensorflow.keras.utils import to_categorical

meals = []
meals += [mapper.index(input("請問你前天吃什麼："))]
meals += [mapper.index(input("請問你昨天吃什麼："))]

meals = np.reshape(to_categorical(meals, dataset.shape[1]), (1, win_size, dataset.shape[1]))
meal_today = np.argmax(model.predict(meals), axis=-1)
print("我猜你今天會吃：", mapper[meal_today[0]])

請問你前天吃什麼：自助餐
請問你昨天吃什麼：自助餐
1/1 [==============================] - 0s 126ms/step
我猜你今天會吃： 自助餐
